### SIMPLER Bagging Regressor

In [1]:
#Import python libraries/packages

# Pandas to handle dataframes
import pandas as pd    

# Import Scikit-Learn library for the regression model
import sklearn   
from sklearn import preprocessing #sklearn for normalization function
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
#for Bagging regressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

# Import numpy 
import numpy as np

# Import plotting libraries
import matplotlib 
from matplotlib import pyplot as plt


#### Function to do multiple linear regression with bagging regressor (BR)
bagging_regr(test_ratio, n_estim, seed_random, X, y):

In [2]:
def bagging_regr(csv_file, X_user):
    '''
    the function takes in the master csv and user input
    csv_file should be inputted as a string = 'compiled_features_complete.csv'
    test ratio, random state and n_estimator are set (from previous ML optimization)
    returning the predicted output based on user input
    '''

    # Open and load dataset
    bacterial_csv = pd.read_csv(csv_file)
    df = pd.DataFrame(data=bacterial_csv)
    
    #assign input (X) /output (y) features
    X= df[['AA_NP','AA_POS', 'AA_POL','AA_NEG', 'MW', 'AROM', 'ISO_E']]
    y= df['LOG2FC']
    
    #Scale input features
    X_arr = X.values #returns a numpy array for X (needed to use the min_max_scaler)

    X_col_names = list(X.columns.values.tolist()) #get column names to then put back into X_norm

    #min-max normalization (rescaling) of input features
    min_max_scaler = preprocessing.MinMaxScaler()
    X_scaled = min_max_scaler.fit_transform(X_arr)
    X_norm= pd.DataFrame(X_scaled)

    #put back the original column names
    X_norm.columns = X_col_names
    
  
    #set Bagging regressor parameters, from ML training: 
    test_ratio = 0.30
    seed_random = 42
    n_estim= 20
    
    X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=test_ratio, random_state=seed_random, shuffle=True)
    #Model is Bagging Regressor, base estimator is Decision Tree regressor
    model = BaggingRegressor(base_estimator=DecisionTreeRegressor(),n_estimators=n_estim, random_state=seed_random)
    model.fit(X_train, y_train)
    #y_testpredict = model.predict(X_test)
       
      
    user_predict = model.predict(X_user)
    
    return user_predict

## Unit Tests for above code

In [11]:
import unittest # will likely already have this import in master test .py file

from functions.py import bagging_regr # also likely redundant if import already included in master test .py file

class BaggingRegrTestCase(unittest.TestCase):

    def test_incomplete_csv(self):
        '''
        This test function is relevant if the master csv is ever updated to include additional data for model training.
        '''
        
        csv_file = 'test_bagging_1.csv' # csv contains one row with empty cells
        test_user_features = [{'AA_NP':0.23,'AA_POS':0.3,'AA_POL':0.9, 'AA_NEG': 0.3, 'MW':0.95, 'AROM':0.891, 'ISO_E': 0.012}]
        X_user = pd.DataFrame(test_user_features)
        
        # check that the function fails when fed csv with NaN cells
        with self.assertRaises(ValueError):
            bagging_regr(csv_file, X_user)
    
    def test_values_csv(self):
        '''
        This test function is relevant if the master csv is ever updated to include additional data for model training.
        '''
        
        csv_file = 'test_bagging_2.csv' # csv contains one row with strings rather than floats
        test_user_features = [{'AA_NP':0.23,'AA_POS':0.3,'AA_POL':0.9, 'AA_NEG': 0.3, 'MW':0.95, 'AROM':0.891, 'ISO_E': 0.012}]
        X_user = pd.DataFrame(test_user_features)
        
        # check that the function fails when fed csv with values other than floats
        with self.assertRaises(ValueError):
            bagging_regr(csv_file, X_user)


if __name__ == '__main__':
    unittest.main()

ModuleNotFoundError: No module named 'functions'

### EXAMPLE USE OF THE FUNCTION:

User input:

In [7]:
user_features = [{'AA_NP':0.23,'AA_POS':0.3,'AA_POL':0.9, 'AA_NEG': 0.3, 'MW':0.95, 'AROM':0.891, 'ISO_E': 0.012}]
X_user = pd.DataFrame(user_features)
X_user

,AA_NP,AA_POS,AA_POL,AA_NEG,MW,AROM,ISO_E
0,0.23,0.3,0.9,0.3,0.95,0.891,0.012


In [8]:
csv_file = 'compiled_features_complete.csv'
pred_input_logFC = bagging_regr(csv_file, X_user)
print("The predicted log2FC for user input is %.5f" % pred_input_logFC)

The predicted log2FC for user input is 0.20913
